In [1]:
#!pip install pyLDAvis

from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
from gensim import corpora
from gensim.summarization.summarizer import summarize

import pandas as pd
import nltk
import gensim

In [2]:
# 기사 번호
ARTICLE_NO = 1

# 불용어 다운로드
nltk.download('stopwords')

documents = fetch_20newsgroups(shuffle=True, remove=('headers', 'footers', 'quotes')).data
print(f'Got {len(documents)} samples.')
print(documents[ARTICLE_NO])

news_df = pd.DataFrame({'document': documents})

# 알파벳이 아닌 모든 문자를 제거합니다.
news_df['clean_doc'] = news_df['document'].str.replace(r'[^a-zA-Z]', ' ')
# 길이가 3 초과인 단어만 가져옵니다.
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 소문자로 변환합니다.
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())
news_df['clean_doc'][ARTICLE_NO]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nsun5\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Got 11314 samples.
A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.


C:\Users\nsun5\AppData\Local\Temp/ipykernel_2276/3890180119.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace(r'[^a-zA-Z]', ' ')


'fair number brave souls upgraded their clock oscillator have shared their experiences this poll please send brief message detailing your experiences with procedure speed attained rated speed cards adapters heat sinks hour usage floppy disk functionality with floppies especially requested will summarizing next days please network knowledge base have done clock upgrade haven answered this poll thanks'

In [3]:
# 불용어를 받아오고 토큰화시킨 뒤, 불용어를 제거합니다.
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
print(tokenized_doc[ARTICLE_NO])

# 뉴스에서의 단어 인베딩과 빈도수를 기록합니다.
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[ARTICLE_NO]) 

['fair', 'number', 'brave', 'souls', 'upgraded', 'clock', 'oscillator', 'shared', 'experiences', 'poll', 'please', 'send', 'brief', 'message', 'detailing', 'experiences', 'procedure', 'speed', 'attained', 'rated', 'speed', 'cards', 'adapters', 'heat', 'sinks', 'hour', 'usage', 'floppy', 'disk', 'functionality', 'floppies', 'especially', 'requested', 'summarizing', 'next', 'days', 'please', 'network', 'knowledge', 'base', 'done', 'clock', 'upgrade', 'answered', 'poll', 'thanks']
[(24, 2), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 2), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 2), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1)]


In [4]:
# 20개의 토픽, k=20
NUM_TOPICS = 20
# corpus를 가지고 lda모델 학습 num_topics은 토픽의 개수 , passes는 알고리즘의 동작 횟수
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
# 학습한 lda모델의 토픽별 핵심 단어 확인, num_words 는 단어의 개수
topics = ldamodel.print_topics(num_words=4)
# 결과 출력
for topic in topics:
    print(topic)

(0, '0.014*"price" + 0.013*"sale" + 0.010*"jpeg" + 0.010*"offer"')
(1, '0.012*"would" + 0.009*"time" + 0.009*"think" + 0.008*"like"')
(2, '0.017*"window" + 0.015*"file" + 0.014*"server" + 0.014*"files"')
(3, '0.009*"information" + 0.007*"public" + 0.007*"president" + 0.006*"encryption"')
(4, '0.011*"people" + 0.010*"jesus" + 0.006*"said" + 0.006*"bible"')
(5, '0.013*"medical" + 0.012*"science" + 0.012*"health" + 0.010*"disease"')
(6, '0.022*"windows" + 0.018*"card" + 0.016*"software" + 0.012*"video"')
(7, '0.013*"team" + 0.013*"game" + 0.013*"year" + 0.008*"games"')
(8, '0.009*"believe" + 0.009*"people" + 0.009*"would" + 0.008*"think"')
(9, '0.034*"file" + 0.024*"entry" + 0.016*"output" + 0.015*"program"')
(10, '0.026*"water" + 0.015*"libxmu" + 0.012*"symbol" + 0.009*"doug"')
(11, '0.023*"would" + 0.019*"know" + 0.017*"like" + 0.013*"anyone"')
(12, '0.031*"wire" + 0.030*"ground" + 0.021*"wiring" + 0.021*"neutral"')
(13, '0.013*"gordon" + 0.013*"banks" + 0.012*"pitt" + 0.012*"baltimore"

In [20]:
article = '''

   학교 문법에서는 ‘문장이 중의성을 갖는 경우가 있으니, 이를 최대한 피해야 한다.’라는 식으로 교육하며, 이러한 중의성이 왜 일어나는지는 자세히 알려주지 않는다. 이론언어학 중 통사론을 통해 그 해답을 찾을 수 있었다. 문장의 구절구조규칙에 따라 중의성이 발생하는 경우가 생긴다. 단어의 배열은 같아도, 문장의 구절구조를 어떻게 분석하느냐에 따라 뜻이 달라지게 된다. 문장의 중의성은 심층 구조와 표면 구조가 다르기 때문에 발생한다.
  반대의 경우도 존재한다. 표면 구조가 다르지만 심층 구조가 같은 경우다. 영어에서 일부 5형식 문장을 that절을 이끄는 3형식 문장으로 변환할 수 있다. 수형도로 나타내면 하단에 있는 NP와 VP가 묶여 위로 올라가게 되는데, 이것을 인상변형이라고 한다. 이러한 이동 변형을 통해 같은 심층 구조를 가진(같은 의미를 가진) 여러 다른 형태의 문장(표면 구조가 다른 문장)을 만들 수 있다.

  자연어 처리는 컴퓨터가 자연어를 이해하도록 하는 것이다. 이를 위해선 언어를 형태소 단위로 분리하는 작업이 필요하다. 그러나 자연어에서는 수많은 불규칙 활용이 존재하며, 이들은 유한상태변환기를 통해 단어의 원형을 찾아야 한다. 전산형태론에서는 규칙을 기반으로 원형을 복원하는 방법과 의미를 기반으로 원형을 복원하는 두 가지 방법이 있다.
  컴퓨터는 문자를 전산화하기 위해 문자를 2진수로 변환하는 방법인 인코딩을 통해 표현한다. ASCII, EUC-KR, UTF-8 등 다양한 인코딩 방식이 있는데 현재는 UTF-8이 사실상 표준으로 자리잡으며 대부분의 문자를 효과적으로 표현할 수 있게 되었다.

컴퓨터에는 직관이란 개념이 존재하지 않기에 모든 것을 규정해서 풀어나가야 한다. 인공지능의 발전과 비교했을 때 자연어 처리는 너무나도 더디다. 그만큼 인간의 언어 체계가 상당히 복잡하다는 뜻이 아닐까 하는 생각이 든다.
  친구들과 탐구 과정을 공유하며 언어학 및 언어학의 주변 분야에 대한 시각을 길렀다. 언어학의 탐구 태도, 한시의 언어적 표현 방식, 서양의 라틴어와 동양의 한문 입지, 독일 미학과 문법 및 언어 번역의 한계점, 다언어 국가의 사회적 문제와 분쟁 해결을 위한 노력 등에 대해 공유했다. 한 언어에서 다른 언어로 완전한 번역이 가능한지, 지역 언어의 교육 축소는 합리적인지에 생각해보는 등 언어학 그 자체뿐만 아니라 그것과 연계된 다양한 것들에 대한 시각을 넓힐 수 있었다.

'''

print(summarize(article, word_count = 70))

문장의 구절구조규칙에 따라 중의성이 발생하는 경우가 생긴다.
이러한 이동 변형을 통해 같은 심층 구조를 가진(같은 의미를 가진) 여러 다른 형태의 문장(표면 구조가 다른 문장)을 만들 수 있다.
친구들과 탐구 과정을 공유하며 언어학 및 언어학의 주변 분야에 대한 시각을 길렀다.
한 언어에서 다른 언어로 완전한 번역이 가능한지, 지역 언어의 교육 축소는 합리적인지에 생각해보는 등 언어학 그 자체뿐만 아니라 그것과 연계된 다양한 것들에 대한 시각을 넓힐 수 있었다.
